In [ ]:
import re
from typing import Tuple
from functools import cache

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
blueprints = []
for l in lines:
    blueprint_text = l.split(":")[0]
    blueprint = int(re.findall(r"(\d+)", blueprint_text)[0])

    robot_texts = l.split(":")[1].strip().split(".")
    robot_costs = []

    ore_robot_text = robot_texts[0]
    ore_costs = int(re.findall(r"(\d+)", ore_robot_text)[0])
    robot_costs.append((ore_costs, 0, 0, 0))

    clay_robot_text = robot_texts[1]
    clay_costs = int(re.findall(r"(\d+)", clay_robot_text)[0])
    robot_costs.append((clay_costs, 0, 0, 0))

    obsidian_robot_text = robot_texts[2]
    obsidian_costs = [int(c) for c in re.findall(r"(\d+)", obsidian_robot_text)]
    robot_costs.append((obsidian_costs[0], obsidian_costs[1], 0, 0))

    geode_robot_text = robot_texts[3]
    geode_costs = [int(c) for c in re.findall(r"(\d+)", geode_robot_text)]
    robot_costs.append((geode_costs[0], 0, geode_costs[1], 0))

    blueprints.append((blueprint, tuple(robot_costs)))

blueprints[:3]

In [ ]:
def add_tuples(
    tuple1: Tuple[int, int, int, int], tuple2: Tuple[int, int, int, int]
) -> Tuple[int, int, int, int]:
    res_tuple = tuple(map(lambda i, j: i + j, tuple1, tuple2))
    return res_tuple


def subtract_tuples(
    tuple1: Tuple[int, int, int, int], tuple2: Tuple[int, int, int, int]
) -> Tuple[int, int, int, int]:
    res_tuple = tuple(map(lambda i, j: i - j, tuple1, tuple2))
    return res_tuple


def can_build(bp, inventory):
    return all(map(lambda i, j: i <= j, bp, inventory))

In [ ]:
@cache
def search_max_geodes(
    blueprint,
    robots: Tuple[int, int, int, int],
    inventory: Tuple[int, int, int, int],
    minutes,
    max_minutes,
):
    global current_max_geodes
    global current_max_geodes_minutes

    if minutes == max_minutes:
        # print(inventory, robots)
        return inventory[3]

    if inventory[3] > 0 and minutes <= current_max_geodes_minutes:
        current_max_geodes = max(current_max_geodes, inventory[3])
        current_max_geodes_minutes = minutes

    if minutes > current_max_geodes_minutes and inventory[3] < current_max_geodes:
        # print("prune")
        return 0

    ore_costs = blueprint[0]
    max_ore_cost = max(
        blueprint[0][0], blueprint[1][0], blueprint[2][0], blueprint[3][0]
    )
    clay_costs = blueprint[1]
    max_clay_costs = max(
        blueprint[0][1], blueprint[1][1], blueprint[2][1], blueprint[3][1]
    )
    obsidian_costs = blueprint[2]
    max_obsidian_costs = max(
        blueprint[0][2], blueprint[1][2], blueprint[2][2], blueprint[3][2]
    )
    geode_costs = blueprint[3]

    prod_inventory = add_tuples(robots, inventory)
    # print(minutes, inventory, robots, prod_inventory)
    robot_build_options = []

    # build geode
    built_geode = False
    if can_build(geode_costs, inventory):
        # print("Build geode")
        new_inventory = subtract_tuples(prod_inventory, geode_costs)
        new_robots = add_tuples(robots, (0, 0, 0, 1))
        robot_build_options.append(
            search_max_geodes(
                blueprint, new_robots, new_inventory, minutes + 1, max_minutes
            )
        )
        built_geode = True

    built_obsidian = False
    if (
        not built_geode
        and can_build(obsidian_costs, inventory)
        and robots[2] < max_obsidian_costs
    ):
        # print("Build obsidian")
        new_inventory = subtract_tuples(prod_inventory, obsidian_costs)
        new_robots = add_tuples(robots, (0, 0, 1, 0))
        robot_build_options.append(
            search_max_geodes(
                blueprint, new_robots, new_inventory, minutes + 1, max_minutes
            )
        )
        built_obsidian = True

    if (
        not built_geode
        and not built_obsidian
        and can_build(clay_costs, inventory)
        and robots[1] < max_clay_costs
    ):
        # print("Build clay")
        new_inventory = subtract_tuples(prod_inventory, clay_costs)
        new_robots = add_tuples(robots, (0, 1, 0, 0))
        robot_build_options.append(
            search_max_geodes(
                blueprint, new_robots, new_inventory, minutes + 1, max_minutes
            )
        )

    if (
        not built_geode
        and not built_obsidian
        and can_build(ore_costs, inventory)
        and robots[0] < max_ore_cost
    ):
        # print("Build ore")
        new_inventory = subtract_tuples(prod_inventory, ore_costs)
        new_robots = add_tuples(robots, (1, 0, 0, 0))
        robot_build_options.append(
            search_max_geodes(
                blueprint, new_robots, new_inventory, minutes + 1, max_minutes
            )
        )

    # do nothing
    if not built_geode and not built_obsidian:
        robot_build_options.append(
            search_max_geodes(
                blueprint, robots, prod_inventory, minutes + 1, max_minutes
            )
        )

    return max(robot_build_options)


blueprint = blueprints[0][1]
current_max_geodes = 0
current_max_geodes_minutes = 25
search_max_geodes(blueprint, (1, 0, 0, 0), (0, 0, 0, 0), 0, 24)

In [ ]:
current_max_geodes = 0
current_max_geodes_minutes = 25

res = 0
for bp in blueprints:
    current_max_geodes = 0
    current_max_geodes_minutes = 25
    bp_num = bp[0]
    blueprint = bp[1]
    # print(blueprint)

    current_max_geodes = 0
    current_max_geodes_minutes = 25

    max_geodes = search_max_geodes(blueprint, (1, 0, 0, 0), (0, 0, 0, 0), 0, 24)
    search_max_geodes.cache_clear()
    # print(bp_num, max_geodes)
    res += bp_num * max_geodes

res

In [ ]:
current_max_geodes = 0
current_max_geodes_minutes = 25

res = 1
for bp in blueprints[:3]:
    current_max_geodes = 0
    current_max_geodes_minutes = 25
    bp_num = bp[0]
    blueprint = bp[1]
    # print(blueprint)

    current_max_geodes = 0
    current_max_geodes_minutes = 25

    max_geodes = search_max_geodes(blueprint, (1, 0, 0, 0), (0, 0, 0, 0), 0, 32)
    search_max_geodes.cache_clear()
    # print(bp_num, max_geodes)
    res *= max_geodes

res